# Recursive Linked Data Structures

# Binary Search Trees

The **Binary Search Tree (BST)** is a data structure for storing $n$ items. We've already seen a few different ways to store a bunch of items.  Let's compare them:
* Like linked lists, a BST is a linked data structure: it consists of $n$ nodes with links (pointers/references) between them.  Unlike linked lists, BST nodes have multiple links, organizing the data recursively instead of linearly.  This enables BSTs to access a desired item much faster than the linear scan required by a linked list.
* Like arrays (Python `list`), a BST effectively maintains items in an order, and like sorted arrays, BSTs support fast binary search.  Unlike arrays (and like linked lists), BSTs support efficient insertion and deletion of items in the middle of the order.
* Like hash tables (Python `dict`), a BST supports efficient searching for a given key (and finding a corresponding value), though BSTs are not quite as efficient.  Whereas hash tables take roughly constant time per search/insert/delete, a BST takes time proportional to the "height" $h$ of the BST.  Typically, $h$ grows logarithmically in the number $n$ of nodes, but guaranteeing this is not easy and will be left to 6.006.
* Unlike hash tables (Python `dict`), when you search for a key that isn't in the data structure, a BST lets you to find the next closest keys that are in the data structure. This is called "inexact search".

## BSTs are like dynamic binary search

The high performance of BSTs comes from the efficiency of **binary search**.  Remember binary search?  By storing $n$ keys in sorted order, binary search performs a search by looking at roughly $\log_2 n$ keys. Also, even if the search key isn't in the array, it lets you find the next closest keys.

In [1]:
def binary_search(A, x):
    """If A is sorted, finds the index of x or the next largest item."""
    # Focus on A[i], ..., A[j] - initially the whole array
    i = 0
    j = len(A)-1
    while i <= j:
        m = (i + j) // 2  # middle index
        if A[m] == x:
            return m
        elif A[m] < x:
            i = m+1
        else:
            j = m-1
    return i  # successor of x; j would give predecessor of x

assert binary_search([1, 3, 4, 5, 9, 11, 23], 5) == 3
assert binary_search([1, 3, 4, 5, 9, 11, 23], 4.5) == 3
assert binary_search([1, 3, 4, 5, 9, 11, 23], 4) == 2
assert binary_search([1, 3, 4, 5, 9, 11, 23], 6) == 4

The trouble with binary search is that it's hard to add or remove keys from a sorted array.  As we discussed around linked lists, inserting or deleting in the middle of an array requires linear time to shift the items around.

But we can still get inspiration from binary search.  Let's think about the sequence of comparisons that binary search makes when exploring a sorted array:

<img src="img/binary_search_tree.svg">

We can turn this picture into a linked data structure consisting of nodes that link to two nodes: where to go when you find a key smaller or greater than the node's key.  In mathematics, this structure is called a **rooted binary tree**: it has one special node called the root (assuming it has any nodes), and two "children" distinguished as left child and right child (each of which can be null).

In [2]:
class BST:
    def __init__(self, key, value, left=None, right=None):
        self.key = key
        self.value = value
        self.left = left
        self.right = right

def example():
    return BST(5, 'D',
        BST(3, 'B',
           BST(1, 'A'),
           BST(4, 'C')
        ),
        BST(11, 'F',
           BST(9, 'E'),
           BST(23, 'G')
        )
    )

print(example())

## Rendering to string

Before we implement any operations on BSTs, let's make it easy to visualize a BST.  We'll implement a `__str__` method which defines how to render an object into a string.  Python's built-in function `print` calls Python's built-in function `str` which ends up calling the `__str__` method when it exists, so this will override the default `print` behavior.

Because a BST is a recursive linked structure, `__str__` is naturally implemented recursively.  The base case is an empty tree (represented by `None`) and the recursive case is a nonempty tree.  In this case, it's helpful to define a helper function for the recursion.  Why?  Because we'd like to be able to recurse on `None` values, but we can't override `None.__str__` to do what we want.  By using a regular function `recurse` instead of a method `__str__`, we can define what to do with `None` values.

(For the purpose of this notebook, we're defining this method using inheritence and overriding the previous definition of the `BST` class.  In normal code, you wouldn't do this; instead, you'd define all the methods in a single `class BST` definition.  See `bst.py`.)

In [3]:
class BST(BST):
    def __str__(self):
        def recurse(node, prefix):
            if node is None:
                return f"{prefix}None\n"
            else:
                return f"{prefix}{node.key}: {node.value}\n" + \
                    recurse(node.left, prefix + '  ') + \
                    recurse(node.right, prefix + '  ')
        return recurse(self, '')

print(example())

5: D
  3: B
    1: A
      None
      None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None



## Binary Search in a Binary Search Tree

Now let's write code to follow the binary search diagram to search for a key.  Specifically, we'll write a `get` method that works like `dict.get`: it returns the value associated with a key, or a special value (defaulting to `None`) if the key isn't there.  This method can be naturally written as a recursion (following the recursion of the data structure) or with a loop (because there's only one direction we need to go).

In [4]:
class BST(BST):
    def get(self, key, missing=None):
        """Get the value associated with given key,
        or if key isn't there, return missing."""
        if key == self.key:
            return self.value
        elif key < self.key:
            node = self.left
        else: # key > self.key
            node = self.right
        if node is None:
            return missing
        else:
            return node.get(key, missing)

In [5]:
class BST(BST):
    def get(self, key, missing=None):
        """Get the value associated with given key,
        or if key isn't there, return missing."""
        node = self
        while node is not None:
            if key == node.key:
                return node.value
            elif key < node.key:
                node = node.left
            else: # key > node.key
                node = node.right
        # node is None, so didn't find key
        return missing

In [6]:
assert example().get(0) == None
assert example().get(1) == 'A'
assert example().get(2) == None
assert example().get(3) == 'B'
assert example().get(4) == 'C'
assert example().get(4.5) == None
assert example().get(5) == 'D'

To see what's going on, we can write a similar method to return the path of nodes that get visited while searching for the key:

In [7]:
class BST(BST):
    def path(self, key):
        """Return a list of node values visited along the way to find key"""
        values = []
        node = self
        while node is not None:
            values.append(node.value)
            if key == node.key:
                return values
            elif key < node.key:
                node = node.left
            else: # key > node.key
                node = node.right
        # node is None, so didn't find key
        values.append(None)
        return values

In [8]:
example().path(4.5)

['D', 'B', 'C', None]

In [9]:
example().path(9)

['D', 'F', 'E']

## Inserting a node

The point of BSTs was to make a sorted array dynamic.  So let's see how to actually modify a BST, first by adding a new (key, value) pair.  The main idea is to search for the new key, and when we "fall off the tree" (reach a `None` value), put a new node there.  So the code will be very similar to the `get` method.

A special case is when we find that the new key actually matches an old key.  One option is to create a new node anyway; then `get` will return one of them, but we won't have control over which.  Instead we opt to overwrite the value in that case, the same as how `dict` works with `d[key] = value`.

In [10]:
class BST(BST):
    def insert(self, key, value):
        """Add node with specified key and value into BST,
        or overwrite existing node with specified key,
        and return the node."""
        node = self
        while True:
            if key == node.key:
                node.value = value
                return node
            elif key < node.key:
                if node.left is None:
                    node.left = BST(key, value)
                    return node.left
                node = node.left
            else: # key > node.key:
                if node.right is None:
                    node.right = BST(key, value)
                    return node.right
                node = node.right

In [11]:
for key in [0, 1, 2]:
    print(f"** Example after inserting {key}:")
    tree = example()
    tree.insert(key, 'Z')
    print(tree)

** Example after inserting 0:
5: D
  3: B
    1: A
      0: Z
        None
        None
      None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None

** Example after inserting 1:
5: D
  3: B
    1: Z
      None
      None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None

** Example after inserting 2:
5: D
  3: B
    1: A
      None
      2: Z
        None
        None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None



## dict interface

While we're implementing operations like `dict` does, let's implement methods to make a BST actually look like a `dict`, by supporting `key in bst`, `bst[key]`, and `bst[key] = value` operations.  These operators end up calling special methods `__contains__`, `__getitem__`, and `__setitem__`, respectively, so we just need to implement those methods:

In [12]:
class Missing: pass

class BST(BST):
    def __contains__(self, key):
        # Use unique Missing value to detect absent key
        # (as opposed to a key with an associated value of None)
        value = self.get(key, Missing)
        return value is not Missing
    def __getitem__(self, key):
        value = self.get(key, Missing)
        if value is Missing:
            raise KeyError(key)
        else:
            return value
    def __setitem__(self, key, value):
        self.insert(key, value)

In [13]:
tree = example()
print(1 in tree)
print(tree[1])

True
A


In [14]:
tree[1] = 'AAA'
print(tree[1])

AAA


In [15]:
print(2 in tree)
tree[2] = 'AB'
print(2 in tree)
print(tree[2])

False
True
AB


## In-order traversal

A BST is like a dynamic sorted array in that it supports binary search as well as inserting and deleting keys.  But it's also like a dynamic sorted array in the way that it's really easy to list all of the items in order by key.  The algorithm to do this is called **in-order traversal**, and it's naturally stated recursively:

1. Recurse on the left subtree (if there is one)
2. Output the root node
3. Recurse on the right subtree (if there is one)

The key property that makes this work is the **BST property**: all nodes in the left subtree of a node have smaller keys than the node, and all nodes in the right subtree of a node have larger keys than the node.  If you look at the `insert` code above, you can convince yourself that it preserves this property (assuming it held originally).

In [16]:
class BST(BST):
    def print_in_order(self):
        """Print all (key, value) pairs in order by key"""
        if self.left is not None:
            self.left.print_in_order()
        print(f"{self.key}: {self.value}")
        if self.right is not None:
            self.right.print_in_order()

In [17]:
example().print_in_order()

1: A
3: B
4: C
5: D
9: E
11: F
23: G


What if we want to return a list of all the items in the BST, like `dict.items`, but in order by key?  Like `print_in_order`, this code is naturally expressed recursively.  But now it's helpful to define a helper function `recurse` in particular to give it access to a shared variable to accumulate all the items.  It also lets us more easily deal with `None` values.

In [18]:
class BST(BST):
    def items(self):
        """Return a list of (key, value) pairs in order by key"""
        items = []
        def recurse(node):
            if node is None: return
            recurse(node.left)
            items.append((node.key, node.value))
            recurse(node.right)
        recurse(self)
        return items

In [19]:
example().items()

[(1, 'A'), (3, 'B'), (4, 'C'), (5, 'D'), (9, 'E'), (11, 'F'), (23, 'G')]

This method lets us do easy `for` loops over BSTs!  For example:

In [20]:
total = 0
for key, value in example().items():
    total += key
print("Total key value:", total)

Total key value: 56


## Iterators and Generators

A problem with the code above is that it traverses the entire tree and builds an entire list of all items before doing the summation work.  This is a bit slower and takes a lot more memory (roughly storing an entire copy of the data that will later be thrown away).

### Iterators

Python has **iterators** that let you loop through a structure without having to build and store a list representing what you want to iterate through.  In Python 3, an iterator is an object with a `__next__()` method that returns the next element in the iterator, with successive calls to the same function producing successive elements.  `__next__()` should raise a `StopIterator` exception when it reaches the end of what it wants to list, though you can also define infinite iterators that have no end.

You can turn most objects into iterators using Python's built-in `iter` function:

In [21]:
it = iter([1,2,3])
it

In [22]:
it.__next__()

1

In [23]:
it.__next__()

2

In [24]:
it.__next__()

3

In [25]:
it.__next__()

StopIteration: 

### for loops over iterators

Usually you don't interact with an iterator's `__next__` interface directly.  Instead, you can use a `for` loop to iterate through the items in the iterator:

In [26]:
it = iter([1,2,3])
for x in it:
    print(x)
for x in it:
    print(x, 'again')

1
2
3


Notice that, once an iterator is fully consumed, it's over: there's no way to restart it at the beginning, other than creating another iterator.  You can, however, convert an iterator back to a `list` which would let you iterate over it many times.

In [27]:
list(iter([1,2,3])) == [1,2,3]

True

In fact, `for x in y` always converts `y` into an iterator in order to loop through the elements.  We can modify how this conversion works by specifying an `__iter__` method on our class:

In [28]:
class BST(BST):
    def __iter__(self):
        return iter(self.items())

In [29]:
total = 0
for key, value in example():
    total += key
print("Total key value:", total)

Total key value: 56


### Custom iterators via classes

This is nicer code than before, but it still has the problem that it creates the entire list of items before it can iterate through them.  How do we make efficient iterators?

We can create our own iterator class by implementing a custom `__next__` method (and an `__iter__` method so that it works in a `for` loop), but this is usually quite tedious.  Here's an example of making an iterator for a range like `[1, 2, 3]` that doesn't actually store a list:  (This is roughly how Python 3's built-in `range` function works.)

In [30]:
class IterRange:
    """Single iteration from start to end"""
    def __init__(self, start, end):
        self.value = start
        self.end = end
    def __next__(self):
        value = self.value
        if value < self.end:
            self.value += 1
            return value
        else:
            raise StopIteration()
    def __iter__(self):
        return self  # this object is its own iterator (needed for 'for')

In [31]:
for x in IterRange(1, 4):
    print(x)

1
2
3


### Custom iterators via generators

A much easier way to make iterators is with **generator functions**.  A generator is a function containing a special `yield` statement that specifies the next value to include in the iteration.  For example, here's code to iterate through the values 1, 2, 3:

In [32]:
def OneTwoThree():
    yield 1
    yield 2
    yield 3
    #implicit 'return None' which stops the iteration

In [33]:
OneTwoThree

<function __main__.OneTwoThree()>

In [34]:
OneTwoThree()

<generator object OneTwoThree at 0x000002685758E8C8>

In [35]:
it = OneTwoThree()
for x in it:
    print(x)
for x in it:
    print(x, 'again')

1
2
3


What makes generators so useful is that you can put arbitrary code between `yield` statements, so that it's easy to compute the next value to output, while keeping track of any state (local variables) you need from before.  Here's a generator version of `IterRange` from above:

In [36]:
def IterRange(start, end):
    value = start
    while value < end:
        yield value
        value += 1
    #implicit 'return None' which stops the iteration

In [37]:
it = IterRange(1, 4)
for x in it:
    print(x)
for x in it:
    print(x, 'again')

1
2
3


Effectively, `yield` causes the generator function to *suspend*, spit out the specified value to whatever is iterating it (probably a `for` loop or a call to `__next__`), and then when the iterator gets asked for another value, the generator function *resumes*, picking up where it left off.  Crucially, all local variables (here, `value`, `start`, and `end`) have the same value as they did before, making it easy to figure out the next value to `yield`.

### Generator-based BST iteration

Now let's go back to `BST.__iter__` and try to rewrite it without generating a list, and instead using a generator.  The code is similar to `items` above.  It's a little more complicated because, when we recurse on one of the subtrees, we get an iterator of the results, so we need to iterate through it and `yield` each one to send it to the next level up.

In [38]:
class BST(BST):
    def __iter__(self): # not a generator, but returns one
        """Return an iterator of (key, value) pairs in order by key"""
        def recurse(node): # generator
            if node is None: return
            for x in recurse(node.left):
                yield x
            yield (node.key, node.value)
            for x in recurse(node.right):
                yield x
        return recurse(self)

In [39]:
total = 0
for key, value in example():
    total += key
print("Total key value:", total)

Total key value: 56


Python has a special shorthand for "iterate through this iterator and `yield` every value": `yield from`.  Here's a cleaner version of the above code:

In [40]:
class BST(BST):
    def __iter__(self): # not a generator, but returns one
        """Return an iterator of (key, value) pairs in order by key"""
        def recurse(node): # generator
            if node is None: return
            yield from recurse(node.left)
            yield (node.key, node.value)
            yield from recurse(node.right)
        return recurse(self)

Arguably, this code would be even cleaner without the helper `recurse` function, like how we wrote the original in-order traversal pseudocode and `print_in_order` method.  Notice that `yield from` automatically calls Python's built-in `iter` function which in turn calls our custom `__iter__` method to generate an iterator.

In [41]:
class BST(BST):
    def __iter__(self): # generator
        """Return an iterator of (key, value) pairs in order by key"""
        if self.left is not None:
            yield from self.left
        yield (self.key, self.value)
        if self.right is not None:
            yield from self.right

In [42]:
total = 0
for key, value in example():
    total += key
print("Total key value:", total)

Total key value: 56


## Finding nearby elements (inexact search)

Going back to what we promised out of BSTs, let's see how to find nearby keys when the key we're searching for doesn't exist.  This is something `dict`s can't do (efficiently).  The following code is similar to `get` and `insert` above.

In [43]:
class BST(BST):
    def nearby(self, key):
        """Find a node containing the key, or if it's not present,
        either the next smaller key or the next larger key."""
        node = self
        while True:
            if key == node.key:
                node.value = value
                return node, "exact"
            elif key < node.key:
                if node.left is None:
                    return node, "larger"
                node = node.left
            else: # key > node.key:
                if node.right is None:
                    return node, "smaller"
                node = node.right
    def __repr__(self):
        """Short format for node that doesn't recurse.
        This gets called by tuple.__str__, for example."""
        return f"<BST {self.key}: {self.value}>"

In [44]:
example().nearby(0)

(<BST 1: A>, 'larger')

In [45]:
example().nearby(1)

(<BST 1: G>, 'exact')

In [46]:
example().nearby(2)

(<BST 1: A>, 'smaller')

In [47]:
example().nearby(4.5)

(<BST 4: C>, 'smaller')

## Deletion

Finally, we promised that we could both insert and delete (key, value) pairs from a BST, but we've only seen how to do insert.  Deletion is a little messier.  Like insert, we first need to find the node to delete.  The question is what to do next.  There are a few cases:

1. It's easy enough to delete a node with no children: just unlink it from its parent.

   <img src="img/delete1.svg">

2. It's also not hard to delete a node with only one child, similar to how we deleted a node from a linked list: make its parent link to its unique child instead.

   <img src="img/delete2.svg">

3. The hardest case is deleting a node with two children.  The idea here is to reduce to a previous case we know how to do (Cases 1 or 2).  Specifically, we're going to replace the data in the node `x` we want to delete with the data of the very next node `y` in the in-order traversal, i.e., the key that comes right after this one.  This will clearly preserve the in-order traversal, which is equivalent to saying it preserves the BST property.  But it also happens that `y` is easy to find from `x`, given that `x` has two children: `x.right` has nodes of larger key, and we want the smallest such key, so `y` is the leftmost descendant of `x.right` (possibly `x.right` itself, if it has no left child).

   <img src="img/delete3.svg">
   
   Still, we have to delete `y`.  How can we do it?  Recursively!  `y` is lower in the tree than `x`, so this recursion will eventually terminate (we're consistently decreasing "height" from the bottom of the tree).  In fact, `y` can't have a left child (otherwise it would be a better candidate for "the key right after `x`"), so `y` will actually fall into one of the two cases above.  So we'll recurse at most once.
   
Now that we have the trick for the last case, we can reduce the number of cases.  The trick works provided `x.right` has some nodes, i.e., isn't `None`.  So we can reduce to just two cases: `x.right is None` or not.  Still, the code is pretty long.

In [48]:
class BST(BST):
    def delete(self, key, parent=None):
        """Delete node with specified key, if there is one.
        Return whether a node was deleted."""
        node = self
        while node is not None:
            if key == node.key:
                break
            else:
                parent = node
                if key < node.key:
                    node = node.left
                else: # key > node.key:
                    node = node.right
        if node is None: # key not found
            return False
        
        if node.right is None:
            # Case 1: no right child.
            # We either have a left child or not, and link our parent there.
            if parent.left is node:
                parent.left = node.left
            elif parent.right is node:
                parent.right = node.left
        else:
            # Case 2: right child.
            # Find the very next key, called the successor.
            parent = node
            successor = node.right
            while successor.left is not None:
                parent = successor
                successor = successor.left
            # Replace node's data with successor's data.
            node.key = successor.key
            node.value = successor.value
            # Now delete the successor node, recursively.
            successor.delete(successor.key, parent)

        return True # key found and deleted

    def __delitem__(self, key): # dict iterface
        if not self.delete(key):
            raise KeyError(key)

In [49]:
tree = example()
del tree[0]
print(tree)

KeyError: 0

In [50]:
del tree[5]
print(tree)

9: E
  3: B
    1: A
      None
      None
    4: C
      None
      None
  11: F
    None
    23: G
      None
      None



In [51]:
del tree[11]
print(tree)

9: E
  3: B
    1: A
      None
      None
    4: C
      None
      None
  23: G
    None
    None



In [52]:
del tree[23]
print(tree)

9: E
  3: B
    1: A
      None
      None
    4: C
      None
      None
  None



## Limitations

One important limitation to the code we've presented here is that it's impossible to represent an empty BST, i.e., a BST with zero nodes.  This is because we've chosen to represent a tree by its root node, but if there aren't any nodes, there won't be anything to call an `insert` method on.  As a consequence, it's impossible to delete the last node from a BST:

In [53]:
tree = BST(1, 'A')
del tree[1]

AttributeError: 'NoneType' object has no attribute 'left'

The fix for this problem would be to separate out the notions of BST **nodes** from the overall BST **tree**.  In this new design, the `BST` class would represent a BST, and have a single attribute `root` that points to the root `BSTNode`, if there is one, or `None` if the BST is empty.  The `BSTNode` class would represent a node of the BST, and would have `key`, `value`, `left`, and `right` attributes as above.  This is similar to the linked list design from last lecture, where we had separate `LinkedList` and `Node` classes, and the role of `LinkedList` was to store the `head` pointer.  Then `BST.delete` needs to behave specially when deleting the last node (or when the tree is empty); otherwise, it can call `BSTNode.delete` as implemented above.

We leave writing this alternate design as an exercise to the reader.